In [ ]:
import matplotlib.pyplot as plt
from matplotlib.transforms import Bbox
import geopandas as gpd
import pandas as pd
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import os
from fastprogress.fastprogress import progress_bar
from operator import ge
#pd.set_option('mode.chained_assignment','raise')


In [ ]:
railway = gpd.read_file('../report/figures/all_cities/gis_file/railways_no_duplicate.gpkg')
motorway = gpd.read_file('../report/figures/all_cities/gis_file/motorways_2.gpkg')
regions = gpd.read_file('../data/results/regions/regions.gpkg')


In [ ]:
gemeente = gpd.read_file('../data/raw_data/sociodemographics/Gemeente/WijkBuurtkaart_2017_v3.gpkg', 
                         layer = 'gemeente_2017_v3')
gemeente = gemeente.loc[gemeente['WATER'] == 'NEE']

gemeente['GM_NAAM'] = gemeente['GM_NAAM'].str.replace("'",'')
gemeente['GM_NAAM'] = gemeente['GM_NAAM'].str.replace(' ','_')
gemeente['GM_NAAM'] = gemeente['GM_NAAM'].str.replace('-','_')
gemeente['GM_NAAM'] = gemeente['GM_NAAM'].str.replace('\(','', regex=True)
gemeente['GM_NAAM'] = gemeente['GM_NAAM'].str.replace('.\)','', regex=True)

In [ ]:
# Plotting just the legend.
fig, ax = plt.subplots(1, figsize=(3.3, 3.7))
ax.axis('off')

color_seg = ['#f7d500','#d65800','#a11096']


    # legend
    #patch_title = mpatches.Patch(color = 'None',label='Exposure level:')
    #patch_space = mpatches.Patch(color='None', label='')
    #patch_lvl_1 = mpatches.Patch(color=color_seg[0], label='$< \mu - \sigma^2/3$')
    #patch_lvl_2 = mpatches.Patch(color=color_seg[1], label='$[\mu - \sigma^2/3 , \mu + \sigma^2/3]$')
    #patch_lvl_3 = mpatches.Patch(color=color_seg[2], label='$\geq \mu + \sigma^2/3$')
    #patch_region = mpatches.Patch(facecolor='None',edgecolor = 'black',hatch='\\\\\\', label='Segregated region')
    #patch_water = mpatches.Patch(facecolor='#33c2ff', label='Water')
    #line_railway = mlines.Line2D([], [], color='black', linestyle='--', label='Railway')
    #line_motorway = mlines.Line2D([], [], color='#7a7a7a',label='Highway')

patch_title = mpatches.Patch(color = 'None')
patch_lvl_1 = mpatches.Patch(color=color_seg[0])
patch_lvl_2 = mpatches.Patch(color=color_seg[1])
patch_lvl_3 = mpatches.Patch(color=color_seg[2])
patch_space = mpatches.Patch(color='None')
#patch_lvl_1 = mpatches.Patch(color=color_seg[0], label='$< \mu - \sigma^2/3$')
#patch_lvl_2 = mpatches.Patch(color=color_seg[1], label='$[\mu - \sigma^2/3 , \mu + \sigma^2/3]$')
#patch_lvl_3 = mpatches.Patch(color=color_seg[2], label='$\geq \mu + \sigma^2/3$')
patch_region = mpatches.Patch(facecolor='None',edgecolor = 'black',hatch='\\\\\\')
patch_water = mpatches.Patch(facecolor='#33c2ff')
line_railway_1 = mlines.Line2D([], [], color='white')
line_railway_2 = mlines.Line2D([], [], color='black',linestyle = (0,(3,3)))
#line_railway = mlines.Line2D([], [], color='black', linestyle='--', label='Railway')
line_motorway = mlines.Line2D([], [], color='#7a7a7a')

labels = ['Exposure level:','Low','Middle','High','','Segregated region','Water','Railway','Highway']

ax.legend(handles = [patch_title,patch_lvl_1,patch_lvl_2,patch_lvl_3,patch_space,
                    patch_region,patch_water,(line_railway_1,line_railway_2),line_motorway],
            labels = labels,
            loc = 'upper right',fontsize = 15)

    #ax.legend(handles = [patch_title,patch_lvl_1,patch_lvl_2,patch_lvl_3,patch_space,
    #                 patch_region,patch_water,line_railway,line_motorway],
    #      loc = 'upper right',bbox_to_anchor=(min(aspect_ratio*4/3,2),1))
#    ax.legend(handles = [patch_title,patch_lvl_1,patch_lvl_2,patch_lvl_3,patch_region,patch_water,line_railway,line_motorway],
    #                       loc = 'upper left',ncol = 2,bbox_to_anchor=(0, min(1.4,1/aspect_ratio)))
    #ax.add_artist(ScaleBar(dx =1 ,length_fraction=0.3 *aspect_ratio, location = 'lower left'))

#fig.savefig('../report/figures/all_cities/legend.svg',bbox_inches='tight',pad_inches = 0)
plt.close()

In [ ]:
dir_gemeente = '../data/processed_data/city_boundary/'
dir_geometry_edited = '../data/processed_data/zones_delineation/edited/'
dir_geometry = '../data/processed_data/zones_delineation/'
dir_exposure = '../data/results/exposure/'

list_files = os.listdir(dir_gemeente)
list_files.remove('README_city_boundary.mkd')
list_files.remove('Buitenland.gpkg')
aspect_ratio_city = pd.DataFrame({'city':[],'aspect_ratio':[],'aspect_ratio_fin':[]})

pb = progress_bar(range(len(list_files)))


for i in pb:
    file = list_files[i]
    city = file[:-5]
    pb.comment = city

    gemeente_mask = gemeente.loc[gemeente['GM_NAAM'] == city]

    #if os.path.isfile('../report/figures/all_cities/' + city + '.svg'):
     #   continue

    if city == 'Amsterdam':
        spatial_units_1 = gpd.read_file(dir_geometry_edited + 'PC_' + city + '_1.gpkg')
        spatial_units_2 = gpd.read_file(dir_geometry_edited + 'PC_' + city + '_2.gpkg')
        spatial_units = gpd.GeoDataFrame(pd.concat([spatial_units_1,spatial_units_2],ignore_index=True),
                                         geometry = 'geometry',
                                         crs = 'EPSG:28992')
        exposure_1 = pd.read_csv(dir_exposure + city + '_exposure_1.csv', usecols= ['id_unit','expos_NW'])
        exposure_2 = pd.read_csv(dir_exposure + city + '_exposure_2.csv', usecols= ['id_unit','expos_NW'])
        exposure = pd.concat([exposure_1,exposure_2], ignore_index = True)
    
    elif os.path.isfile(dir_geometry_edited + 'PC_' + file):
        spatial_units = gpd.read_file(dir_geometry_edited + 'PC_' + file)
        exposure = pd.read_csv(dir_exposure + city + '_exposure.csv', usecols= ['id_unit','expos_NW'])

    else:
        spatial_units = gpd.read_file(dir_geometry + 'PC_' + file)
        exposure = pd.read_csv(dir_exposure + city + '_exposure.csv', usecols= ['id_unit','expos_NW'])

    #spatial_units = gpd.read_file('../data/processed_data/zones_delineation/PC_Delft.gpkg')
    spatial_units = spatial_units.drop(columns = ['Postcode','gemeente','area_ratio'])


    spatial_units = spatial_units.merge(exposure, on = 'id_unit')
    average_expos = spatial_units['expos_NW'].mean()
    std_expos = spatial_units['expos_NW'].std()

    # Color system.
    spatial_units['exp_norm'] = 0
    spatial_units['exp_norm'] = (spatial_units['expos_NW']-average_expos)/std_expos
    color_seg = ['#f7d500','#d65800','#a11096']
    spatial_units['color'] = color_seg[1]
    spatial_units['color'] = spatial_units['color'].mask(spatial_units['exp_norm'] <= -1/3,color_seg[0])
    spatial_units['color'] = spatial_units['color'].mask(spatial_units['exp_norm'] > 1/3,color_seg[2])


    # Color system.5
    spatial_units = spatial_units.dissolve(by = 'color').reset_index()
    spatial_units = spatial_units.overlay(gemeente_mask, how = 'intersection',keep_geom_type=True)
    # City boundaries.
    boundaries = spatial_units.dissolve()


    regions_city = regions.loc[(regions['seg'] == 1) & (regions['city'] == city)]
    if (not regions_city.empty) and regions_city.dissolve().area.iloc[0] >= 0.7 * boundaries.area.iloc[0]:
        regions_city.loc[:,:] = None
    else:
        regions_city = regions_city.overlay(boundaries, how = 'intersection',keep_geom_type=True)

    motorway_city = motorway.overlay(boundaries, how = 'intersection',keep_geom_type=True)
    railway_city = railway.overlay(boundaries,how = 'intersection',keep_geom_type=True)

    water_city = gpd.read_file('../../study_1/data/raw_data/water/water_' + file)
    water_city = water_city.dissolve().explode(index_parts=True)
    water_city = water_city.loc[water_city.area>2e5]
    if not water_city.empty:
        water_city = water_city.overlay(boundaries,how = 'intersection',keep_geom_type=True)

    aspect_ratio = (boundaries.bounds['maxy'] - boundaries.bounds['miny']) / \
                (boundaries.bounds['maxx'] - boundaries.bounds['minx'])
    aspect_ratio = aspect_ratio.iloc[0]
        
    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.margins(0)
    spatial_units.plot(ax=ax,color = spatial_units['color'],edgecolor = spatial_units['color'])

    ax.axis('off')

    if (not water_city.empty):
        water_city.plot(ax = ax, color = '#33c2ff')
    if (not motorway_city.empty):
        motorway_city.plot(ax=ax,color = '#7a7a7a')
    if (not railway_city.empty):
        railway_city.plot(ax=ax,color = 'white')
        railway_city.plot(ax=ax,color = 'black',linestyle = (0,(3,3)))
    if (not regions_city.empty):
        regions_city.plot(ax=ax,color = 'none',hatch='\\\\')
        
    boundaries.plot(ax=ax,color = 'none', edgecolor = 'black')

    ax.add_artist(ScaleBar(dx =1 ,length_fraction=0.3, location = 'lower left'))

    fig.savefig('../report/figures/all_cities/' + city + '.svg',bbox_inches='tight',pad_inches = 0)
    aspect_ratio_fin = fig.get_tightbbox(fig.canvas.get_renderer()).bounds[2]/fig.get_tightbbox(fig.canvas.get_renderer()).bounds[3]
    aspect_ratio_city = pd.concat([aspect_ratio_city,pd.DataFrame({'city':[city],
                                                                   'aspect_ratio':[aspect_ratio],
                                                                   'aspect_ratio_fin':[aspect_ratio_fin]})])

    plt.close()

In [ ]:
data_city = pd.read_csv('data.csv')

In [ ]:
data_city['intensity'] = data_city['intensity']*100
data_city['separation'] = data_city['separation']*100
data_city['scale_rel'] = data_city['scale_rel']*100

In [ ]:
data_city = data_city.merge(aspect_ratio_city[['city','aspect_ratio_fin']], left_on = 'GM_NAAM', right_on = 'city')

In [ ]:
data_city = data_city.drop(columns='city')

In [ ]:
data_city.to_csv('data.csv', index = False)